In [22]:
import requests
import json
import dml
import prov.model
import datetime
import uuid
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
contributor = 'mmao95_Dongyihe_weijiang_zhukk'
reads = []
writes = [contributor + '.famous_people']

## exec

In [4]:
startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

True

In [18]:
url = 'http://datamechanics.io/data/us_famous_people.csv'
usfp = pd.read_csv(url)
names = usfp['full_name'].values
    

In [23]:
doc = requests.get("https://www.50states.com/bio/mass.htm").text
soup = bs(doc)
np.append(names, [e.text for e in soup.select("#content b")])

array(['Christopher Columbus', 'Henry Hudson', 'Amerigo Vespucci',
       'John Smith', 'Giovanni da Verrazzano', 'John Muir',
       'Meriwether Lewis and William Clark', 'Sacagawea', 'Kit Carson',
       'Neil Armstrong', 'John Wesley Powell', 'Martin Luther King Jr.',
       'Robert E. Lee', 'Thomas Paine', 'John Brown',
       'Frederick Douglass', 'Susan B. Anthony', 'W.E.B. Du Bois',
       'Tecumseh', 'Sitting Bull', 'Elizabeth Cady Stanton', 'Malcolm X',
       'Abraham Lincoln', 'George Washington', 'Thomas Jefferson',
       'Theodore Roosevelt', 'Ulysses S. Grant', 'Ronald W. Reagan',
       'George W. Bush', 'Franklin Delano Roosevelt', 'Woodrow Wilson',
       'James Madison', 'Andrew Jackson', 'Pocahontas',
       'Eleanor Roosevelt', 'Hillary Clinton', 'Sarah Palin',
       'Martha Washington', 'Helen Keller', 'Sojourner Truth',
       'Jane Addams', 'Edith Wharton', 'Bette Davis', 'Oprah Winfrey',
       'Benedict Arnold', 'Jesse James', 'John Wilkes Booth', 'Al Capone'

In [24]:
list = []
for name in names:
    parts = name.split()
    obj = {
        'full_name': name,
        'first_name': parts[0],
        'last_name': parts[-1] if len(parts) > 1 else None,
        'middle_name': parts[1:-1] if len(parts) > 2 else None
    }
    list.append(obj)

In [25]:
repo.dropCollection('famous_people')
repo.createCollection('famous_people')
repo[writes[0]].insert_many(list)

In [26]:
repo[writes[0]].metadata({'complete': True})
print(repo[writes[0]].metadata())
[record for record in repo[writes[0]].find()]

{'_id': ObjectId('5ca2efdd99d98654c24433a6'), 'complete': True}


[{'_id': ObjectId('5ca2efd499d98654c2443338'),
  'full_name': 'Christopher Columbus',
  'first_name': 'Christopher',
  'last_name': 'Columbus',
  'middle_name': None},
 {'_id': ObjectId('5ca2efd499d98654c2443339'),
  'full_name': 'Henry Hudson',
  'first_name': 'Henry',
  'last_name': 'Hudson',
  'middle_name': None},
 {'_id': ObjectId('5ca2efd499d98654c244333a'),
  'full_name': 'Amerigo Vespucci',
  'first_name': 'Amerigo',
  'last_name': 'Vespucci',
  'middle_name': None},
 {'_id': ObjectId('5ca2efd499d98654c244333b'),
  'full_name': 'John Smith',
  'first_name': 'John',
  'last_name': 'Smith',
  'middle_name': None},
 {'_id': ObjectId('5ca2efd499d98654c244333c'),
  'full_name': 'Giovanni da Verrazzano',
  'first_name': 'Giovanni',
  'last_name': 'Verrazzano',
  'middle_name': ['da']},
 {'_id': ObjectId('5ca2efd499d98654c244333d'),
  'full_name': 'John Muir',
  'first_name': 'John',
  'last_name': 'Muir',
  'middle_name': None},
 {'_id': ObjectId('5ca2efd499d98654c244333e'),
  'full_

In [8]:
repo.logout()
endTime = datetime.datetime.now()
{"start":startTime, "end":endTime}

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Database.logout() is deprecated
  """Entry point for launching an IPython kernel.


{'start': datetime.datetime(2019, 3, 6, 16, 5, 10, 889052),
 'end': datetime.datetime(2019, 3, 6, 16, 5, 17, 533547)}

## prov

In [9]:
doc = prov.model.ProvDocument()
startTime = None
endTime = None
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

doc.add_namespace('alg', 'http://datamechanics.io/algorithm/') # The scripts are in <folder>#<filename> format.
doc.add_namespace('dat', 'http://datamechanics.io/data/') # The data sets are in <user>#<collection> format.
doc.add_namespace('ont', 'http://datamechanics.io/ontology#') # 'Extension', 'DataResource', 'DataSet', 'Retrieval', 'Query', or 'Computation'.
doc.add_namespace('log', 'http://datamechanics.io/log/') # The event log.
doc.add_namespace('bdp', 'https://www.50states.com/bio/mass.htm')

this_script = doc.agent('alg:'+contributor+'#famous_people', {prov.model.PROV_TYPE:prov.model.PROV['SoftwareAgent'], 'ont:Extension':'py'})
resource = doc.entity('bdp:wc8w-nujj', {'prov:label':'311, Service Requests', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
get_names = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
doc.wasAssociatedWith(get_names, this_script)
doc.usage(get_names, resource, startTime, None,
    {prov.model.PROV_TYPE:'ont:Retrieval',
    'ont:Computation':'Data cleaning'
    }
)

fp = doc.entity('dat:'+contributor+'#famous_people', {prov.model.PROV_LABEL:'Famous People', prov.model.PROV_TYPE:'ont:DataSet'})
doc.wasAttributedTo(fp, this_script)
doc.wasGeneratedBy(fp, get_names, endTime)
doc.wasDerivedFrom(fp, resource, get_names, get_names, get_names)

repo.logout()
doc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Database.logout() is deprecated


<ProvDocument>

In [10]:
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

document
  prefix alg <http://datamechanics.io/algorithm/>
  prefix dat <http://datamechanics.io/data/>
  prefix ont <http://datamechanics.io/ontology#>
  prefix log <http://datamechanics.io/log/>
  prefix bdp <https://www.50states.com/bio/mass.htm>
  
  agent(alg:mmao95_Dongyihe_weijiang_zhukk#famous_people, [prov:type='prov:SoftwareAgent', ont:Extension="py"])
  entity(bdp:wc8w-nujj, [prov:label="311, Service Requests", prov:type="ont:DataResource", ont:Extension="json"])
  activity(log:uuide2575f89-f744-4fe5-9902-d1d92e7034fb, -, -)
  wasAssociatedWith(log:uuide2575f89-f744-4fe5-9902-d1d92e7034fb, alg:mmao95_Dongyihe_weijiang_zhukk#famous_people, -)
  used(log:uuide2575f89-f744-4fe5-9902-d1d92e7034fb, bdp:wc8w-nujj, -, [prov:type="ont:Retrieval", ont:Computation="Data cleaning"])
  entity(dat:mmao95_Dongyihe_weijiang_zhukk#famous_people, [prov:label="Famous People", prov:type="ont:DataSet"])
  wasAttributedTo(dat:mmao95_Dongyihe_weijiang_zhukk#famous_people, alg:mmao95_Dongyihe_weij